In [9]:
import os
import pandas as pd
import numpy as np

In [2]:
%pwd

'c:\\Users\\harik\\OneDrive\\Desktop\\HARIKRISHNAN_DETAILS\\Real_Estate_Predictor_Web_App\\Real_Estate_Price_Predictor_Web_App\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\harik\\OneDrive\\Desktop\\HARIKRISHNAN_DETAILS\\Real_Estate_Predictor_Web_App\\Real_Estate_Price_Predictor_Web_App'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PredictConfig:
    date_time_handler_model_file: Path
    log_transformer_model_file: Path
    ordinal_encoder_model_file: Path
    nominal_encoder_model_file: Path
    rare_categorical_handler_file: Path
    remove_outlier_model_file: Path
    feature_scaling_model: Path
    best_model_directory: Path


In [ ]:
from real_estate_price_predictor.constants import *
from real_estate_price_predictor.utils.common import read_yaml, create_directories, save_object, load_object

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_predict_config(self) -> PredictConfig:
        config = self.config.predict

        predict_config = PredictConfig(
            date_time_handler_model_file = config.date_time_handler_model_file ,
            log_transformer_model_file = config.log_transformer_model_file,
            ordinal_encoder_model_file = config.ordinal_encoder_model_file,
            nominal_encoder_model_file = config.nominal_encoder_model_file,
            rare_categorical_handler_file = config.rare_categorical_handler_file,
            remove_outlier_model_file = config.remove_outlier_model_file,
            feature_scaling_model = config.feature_scaling_model,
            best_model_directory = config.best_model_directory
        )

        return predict_config


In [ ]:
class Predict:
    
    def __init__(self, config = PredictConfig):
        self.config = config
    
    def predict(self,dataframe:pd.DataFrame):
        date_time_transformer = load_object(self.config.date_time_handler_model_file)
        log_transformer = load_object(self.config.log_transformer_model_file)
        rare_categorical_model = load_object(self.config.rare_categorical_handler_file)
        ordinal_encoder = load_object(self.config.ordinal_encoder_model_file)
        nominal_encoder = load_object(self.config.nominal_encoder_model_file)
        remove_outlier_model = load_object(self.config.remove_outlier_model_file)
        feature_scaler = load_object(self.config.feature_scaling_model)
        for i in os.listdir(self.config.best_model_directory):
            if i.__contains__('pkl'):
                model_name = i
        model = load_object(os.path.join(self.config.best_model_directory,model_name))
        dataframe = date_time_transformer.transform(dataframe)
        dataframe = log_transformer.transform(dataframe)
        dataframe = rare_categorical_model.transform(dataframe)
        dataframe = ordinal_encoder.transform(dataframe)
        dataframe = nominal_encoder.transform(dataframe)
        dataframe = remove_outlier_model.transform(dataframe)
        dataframe.drop(['YrSold'],axis=1,inplace= True)
        dataframe = pd.DataFrame(feature_scaler.transform(dataframe), columns=dataframe.columns)
        y_pred = model.predict(dataframe)
        return np.exp(y_pred[0])


In [8]:
try:
    config = ConfigurationManager()
    predict_config = config.get_predict_config()
    predict = Predict(config=predict_config)
    result = predict.predict(X_1)
    print(result)
except Exception as e:
    raise e

artifacts/model_trainer\HyperParameterRandomnForrestRegression.pkl
